In this notebook we load a saved dreamer, and run it, to look at params, speed and improve hackability

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# TODO make this a proper package
import os, sys
sys.path.append('..')


from dreamer import parse_args, main, make_env, make_dataset, count_steps,Dreamer

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


Loading textures from cache


In [3]:
# emulate cli
argv = f"../dreamer.py --configs craftax_smaller --logdir ../logdir/craftax_smaller"
argv = argv.split()
print(argv)
config = parse_args(argv)
config

['../dreamer.py', '--configs', 'craftax_smaller', '--logdir', '../logdir/craftax_smaller']
2024-06-07 05:25:00.098 | INFO     | dreamer:parse_args:392 - args=Namespace(act='SiLU', action_repeat=1, actor={'layers': 2, 'dist': 'onehot', 'entropy': 0.0003, 'unimix_ratio': 0.01, 'std': 'none', 'min_std': 0.1, 'max_std': 1.0, 'temp': 0.1, 'lr': 3e-05, 'eps': 1e-05, 'grad_clip': 100.0, 'outscale': 1.0}, batch_length=32, batch_size=256, compile=True, cont_head={'layers': 2, 'loss_scale': 1.0, 'outscale': 1.0}, critic={'layers': 2, 'dist': 'symlog_disc', 'slow_target': True, 'slow_target_update': 1, 'slow_target_fraction': 0.02, 'lr': 3e-05, 'eps': 1e-05, 'grad_clip': 100.0, 'outscale': 0.0}, dataset_size=20000, debug=False, decoder={'mlp_keys': 'state_inventory', 'cnn_keys': 'state_map', 'act': 'SiLU', 'norm': True, 'cnn_depth': 16, 'kernel_size': 4, 'minres': 2, 'mlp_layers': 2, 'mlp_units': 8, 'cnn_sigmoid': False, 'image_dist': 'mse', 'vector_dist': 'symlog_mse', 'outscale': 1.0}, determin

Namespace(act='SiLU', action_repeat=1, actor={'layers': 2, 'dist': 'onehot', 'entropy': 0.0003, 'unimix_ratio': 0.01, 'std': 'none', 'min_std': 0.1, 'max_std': 1.0, 'temp': 0.1, 'lr': 3e-05, 'eps': 1e-05, 'grad_clip': 100.0, 'outscale': 1.0}, batch_length=32, batch_size=256, compile=True, cont_head={'layers': 2, 'loss_scale': 1.0, 'outscale': 1.0}, critic={'layers': 2, 'dist': 'symlog_disc', 'slow_target': True, 'slow_target_update': 1, 'slow_target_fraction': 0.02, 'lr': 3e-05, 'eps': 1e-05, 'grad_clip': 100.0, 'outscale': 0.0}, dataset_size=20000, debug=False, decoder={'mlp_keys': 'state_inventory', 'cnn_keys': 'state_map', 'act': 'SiLU', 'norm': True, 'cnn_depth': 16, 'kernel_size': 4, 'minres': 2, 'mlp_layers': 2, 'mlp_units': 8, 'cnn_sigmoid': False, 'image_dist': 'mse', 'vector_dist': 'symlog_mse', 'outscale': 1.0}, deterministic_run=False, device='cuda:0', disag_action_cond=False, disag_layers=4, disag_log=True, disag_models=10, disag_offset=1, disag_target='stoch', disag_units=

In [25]:
yaml.safe_load(
    pathlib.Path("/media/wassname/SGIronWolf/projects5/worldmodels/dreamerv3-torch/configs.yaml").read_text()
)


/tmp/ipykernel_20867/1136853903.py:1: UnsafeLoaderWarning: 
The default 'Loader' for 'load(stream)' without further arguments can be unsafe.
Use 'load(stream, Loader=ruamel.yaml.Loader)' explicitly if that is OK.
Alternatively include the following in your code:

  import warnings
  warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)

In most other cases you should consider using 'safe_load(stream)'
  yaml.load(


{'defaults': {'logdir': None,
  'traindir': None,
  'evaldir': None,
  'offline_traindir': '',
  'offline_evaldir': '',
  'seed': 0,
  'deterministic_run': False,
  'steps': 1000000.0,
  'parallel': False,
  'eval_every': 10000.0,
  'eval_episode_num': 10,
  'log_every': 10000.0,
  'reset_every': 0,
  'device': 'cuda:0',
  'compile': True,
  'precision': 32,
  'debug': False,
  'video_pred_log': False,
  'task': 'dmc_walker_walk',
  'size': [64, 64],
  'envs': 1,
  'action_repeat': 2,
  'time_limit': 1000,
  'grayscale': False,
  'prefill': 2500,
  'reward_EMA': True,
  'dyn_hidden': 512,
  'dyn_deter': 512,
  'dyn_stoch': 32,
  'dyn_discrete': 32,
  'dyn_rec_depth': 1,
  'dyn_mean_act': 'none',
  'dyn_std_act': 'sigmoid2',
  'dyn_min_std': 0.1,
  'grad_heads': ['decoder', 'reward', 'cont'],
  'units': 512,
  'act': 'SiLU',
  'norm': True,
  'encoder': {'mlp_keys': '$^',
   'cnn_keys': 'image',
   'act': 'SiLU',
   'norm': True,
   'cnn_depth': 32,
   'kernel_size': 4,
   'minres': 4,


In [41]:
# load config, using relative path
import pathlib
import ruamel.yaml as yaml
import tools

# root_dir = pathlib.Path(__file__).parent
configs = yaml.safe_load(
    pathlib.Path("/media/wassname/SGIronWolf/projects5/worldmodels/dreamerv3-torch/configs.yaml").read_text()
)

def recursive_update(base, update):
    for key, value in update.items():
        if isinstance(value, dict) and key in base:
            recursive_update(base[key], value)
        else:
            base[key] = value

name_list = ["defaults"]
defaults = {}
for name in name_list:
    recursive_update(defaults, configs[name])
defaults2 = {k:tools.args_type(v)(v) for k, v in defaults.items()}
config2 = argparse.Namespace(**defaults2)
defaults2

import argparse
parser = argparse.ArgumentParser()
for key, value in sorted(defaults.items(), key=lambda x: x[0]):
    arg_type = tools.args_type(value)
    parser.add_argument(f"--{key}", type=arg_type, default=arg_type(value))
args = parser.parse_args([])
# logger.debug(f"{parser}")

# from loguru import logger
# parser.print_help()
# ;


usage: ipykernel_launcher.py [-h] [--act ACT] [--action_repeat ACTION_REPEAT]
                             [--actor ACTOR] [--batch_length BATCH_LENGTH]
                             [--batch_size BATCH_SIZE] [--compile COMPILE]
                             [--cont_head CONT_HEAD] [--critic CRITIC]
                             [--dataset_size DATASET_SIZE] [--debug DEBUG]
                             [--decoder DECODER]
                             [--deterministic_run DETERMINISTIC_RUN]
                             [--device DEVICE]
                             [--disag_action_cond DISAG_ACTION_COND]
                             [--disag_layers DISAG_LAYERS]
                             [--disag_log DISAG_LOG]
                             [--disag_models DISAG_MODELS]
                             [--disag_offset DISAG_OFFSET]
                             [--disag_target DISAG_TARGET]
                             [--disag_units DISAG_UNITS] [--discount DISCOUNT]
                        

''

In [29]:
# diff two dics
def dict_diff(dict1, dict2):
    diff = {
        'added': {k: dict2[k] for k in dict2 if k not in dict1},
        'removed': {k: dict1[k] for k in dict1 if k not in dict2},
        'modified': {k: (dict1[k], dict2[k]) for k in dict1 if k in dict2 and dict1[k] != dict2[k]}
    }
    return diff

dict_diff(defaults, defaults2)

{'added': {},
 'removed': {},
 'modified': {'size': ([64, 64], (64, 64)),
  'grad_heads': (['decoder', 'reward', 'cont'],
   ('decoder', 'reward', 'cont'))}}

In [16]:
config

Namespace(act='SiLU', action_repeat=1, actor={'layers': 2, 'dist': 'onehot', 'entropy': 0.0003, 'unimix_ratio': 0.01, 'std': 'none', 'min_std': 0.1, 'max_std': 1.0, 'temp': 0.1, 'lr': 3e-05, 'eps': 1e-05, 'grad_clip': 100.0, 'outscale': 1.0}, batch_length=32, batch_size=256, compile=True, cont_head={'layers': 2, 'loss_scale': 1.0, 'outscale': 1.0}, critic={'layers': 2, 'dist': 'symlog_disc', 'slow_target': True, 'slow_target_update': 1, 'slow_target_fraction': 0.02, 'lr': 3e-05, 'eps': 1e-05, 'grad_clip': 100.0, 'outscale': 0.0}, dataset_size=20000, debug=False, decoder={'mlp_keys': 'state_inventory', 'cnn_keys': 'state_map', 'act': 'SiLU', 'norm': True, 'cnn_depth': 16, 'kernel_size': 4, 'minres': 2, 'mlp_layers': 2, 'mlp_units': 8, 'cnn_sigmoid': False, 'image_dist': 'mse', 'vector_dist': 'symlog_mse', 'outscale': 1.0}, deterministic_run=False, device='cuda:0', disag_action_cond=False, disag_layers=4, disag_log=True, disag_models=10, disag_offset=1, disag_target='stoch', disag_units=

In [15]:
config2

Namespace(logdir=None, traindir=None, evaldir=None, offline_traindir='', offline_evaldir='', seed=0, deterministic_run=False, steps='1e6', parallel=False, eval_every='1e4', eval_episode_num=10, log_every='1e4', reset_every=0, device='cuda:0', compile=True, precision=32, debug=False, video_pred_log=False, task='dmc_walker_walk', size=[64, 64], envs=1, action_repeat=2, time_limit=1000, grayscale=False, prefill=2500, reward_EMA=True, dyn_hidden=512, dyn_deter=512, dyn_stoch=32, dyn_discrete=32, dyn_rec_depth=1, dyn_mean_act='none', dyn_std_act='sigmoid2', dyn_min_std=0.1, grad_heads=['decoder', 'reward', 'cont'], units=512, act='SiLU', norm=True, encoder={'mlp_keys': '$^', 'cnn_keys': 'image', 'act': 'SiLU', 'norm': True, 'cnn_depth': 32, 'kernel_size': 4, 'minres': 4, 'mlp_layers': 5, 'mlp_units': 1024, 'symlog_inputs': True}, decoder={'mlp_keys': '$^', 'cnn_keys': 'image', 'act': 'SiLU', 'norm': True, 'cnn_depth': 32, 'kernel_size': 4, 'minres': 4, 'mlp_layers': 5, 'mlp_units': 1024, 'c

In [4]:
from loguru import logger
from tqdm.auto import tqdm
import pathlib

import torch
from torch import nn
from torch import distributions as torchd

import exploration as expl
import models
import tools
import envs.wrappers as wrappers
from parallel import Parallel, Damy

# from main
tools.set_seed_everywhere(config.seed)
if config.deterministic_run:
    tools.enable_deterministic_run()
logdir = pathlib.Path(config.logdir).expanduser()
config.traindir = config.traindir or logdir / "train_eps"
config.evaldir = config.evaldir or logdir / "eval_eps"
config.steps //= config.action_repeat
config.eval_every //= config.action_repeat
config.log_every //= config.action_repeat
config.time_limit //= config.action_repeat

logger.info(f"Logdir {logdir}")
logdir.mkdir(parents=True, exist_ok=True)
config.traindir.mkdir(parents=True, exist_ok=True)
config.evaldir.mkdir(parents=True, exist_ok=True)
step = count_steps(config.traindir)
# step in logger is environmental step
tlogger = tools.Logger(logdir, config.action_repeat * step)
logger.add(logdir/"logger.log")

logger.info("Create envs.")
if config.offline_traindir:
    directory = config.offline_traindir.format(**vars(config))
else:
    directory = config.traindir
train_eps = tools.load_episodes(directory, limit=config.dataset_size)
if config.offline_evaldir:
    directory = config.offline_evaldir.format(**vars(config))
else:
    directory = config.evaldir
eval_eps = tools.load_episodes(directory, limit=1)
make = lambda mode, id: make_env(config, mode, id)
train_envs = [make("train", i) for i in range(config.envs)]
eval_envs = [make("eval", i) for i in range(config.envs)]
if config.parallel:
    train_envs = [Parallel(env, "process") for env in train_envs]
    eval_envs = [Parallel(env, "process") for env in eval_envs]
else:
    train_envs = [Damy(env) for env in train_envs]
    eval_envs = [Damy(env) for env in eval_envs]
acts = train_envs[0].action_space
logger.info(f"Action Space {acts}" )
config.num_actions = acts.n if hasattr(acts, "n") else acts.shape[0]

state = None
if not config.offline_traindir:
    prefill = max(0, config.prefill - count_steps(config.traindir))
    logger.info(f"Prefill dataset ({prefill} steps).")
    if hasattr(acts, "discrete"):
        random_actor = tools.OneHotDist(
            torch.zeros(config.num_actions).repeat(config.envs, 1)
        )
    else:
        random_actor = torchd.independent.Independent(
            torchd.uniform.Uniform(
                torch.Tensor(acts.low).repeat(config.envs, 1),
                torch.Tensor(acts.high).repeat(config.envs, 1),
            ),
            1,
        )

    def random_agent(o, d, s):
        action = random_actor.sample()
        logprob = random_actor.log_prob(action)
        return {"action": action, "logprob": logprob}, None

    state = tools.simulate(
        random_agent,
        train_envs,
        train_eps,
        config.traindir,
        tlogger,
        limit=config.dataset_size,
        steps=prefill,
    )
    tlogger.step += prefill * config.action_repeat
    logger.info(f"Logger: ({tlogger.step} steps).")

logger.info("Simulate agent.")
train_dataset = make_dataset(train_eps, config)
eval_dataset = make_dataset(eval_eps, config)

2024-06-07 03:49:38.306 | INFO     | __main__:<module>:27 - Logdir ../logdir/craftax_smaller
2024-06-07 03:49:38.336 | INFO     | __main__:<module>:36 - Create envs.
2024-06-07 03:50:12.609 | INFO     | __main__:<module>:57 - Action Space Box(0.0, 1.0, (43,), float32)
2024-06-07 03:50:12.611 | INFO     | __main__:<module>:63 - Prefill dataset (0 steps).
2024-06-07 03:50:12.951 | INFO     | __main__:<module>:92 - Logger: (2714 steps).
2024-06-07 03:50:12.952 | INFO     | __main__:<module>:94 - Simulate agent.


In [5]:
train_envs[0].observation_space

/media/wassname/SGIronWolf/projects5/worldmodels/dreamerv3-torch/.venv/lib/python3.9/site-packages/numpy/core/numeric.py:330: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(a, fill_value, casting='unsafe')


Dict('image': Box(0, 255, (130, 110, 3), uint8), 'is_first': Box(0, 0, (1,), uint8), 'is_last': Box(0, 0, (1,), uint8), 'is_terminal': Box(0, 0, (1,), uint8), 'log_achievement_cast_fireball': Box(-inf, inf, (1,), float32), 'log_achievement_cast_iceball': Box(-inf, inf, (1,), float32), 'log_achievement_collect_coal': Box(-inf, inf, (1,), float32), 'log_achievement_collect_diamond': Box(-inf, inf, (1,), float32), 'log_achievement_collect_drink': Box(-inf, inf, (1,), float32), 'log_achievement_collect_iron': Box(-inf, inf, (1,), float32), 'log_achievement_collect_ruby': Box(-inf, inf, (1,), float32), 'log_achievement_collect_sapling': Box(-inf, inf, (1,), float32), 'log_achievement_collect_sapphire': Box(-inf, inf, (1,), float32), 'log_achievement_collect_stone': Box(-inf, inf, (1,), float32), 'log_achievement_collect_wood': Box(-inf, inf, (1,), float32), 'log_achievement_damage_necromancer': Box(-inf, inf, (1,), float32), 'log_achievement_defeat_archer': Box(-inf, inf, (1,), float32), 'l

In [6]:
config = parse_args(argv)
config.num_actions = acts.n if hasattr(acts, "n") else acts.shape[0]
agent = Dreamer(
    train_envs[0].observation_space,
    train_envs[0].action_space,
    config,
    tlogger,
    train_dataset,
).to(config.device)
print(agent)
agent.requires_grad_(requires_grad=False)
if (logdir / "latest.pt").exists():
    checkpoint = torch.load(logdir / "latest.pt")
    agent.load_state_dict(checkpoint["agent_state_dict"])
    tools.recursively_load_optim_state_dict(agent, checkpoint["optims_state_dict"])
    agent._should_pretrain._once = False
    logger.warning(f"Loaded model from {logdir / 'latest.pt'}")

2024-06-07 03:50:13.077 | INFO     | networks:__init__:324 - Encoder CNN shapes: {'state_map': (16, 16, 166)}
2024-06-07 03:50:13.077 | INFO     | networks:__init__:325 - Encoder MLP shapes: {'state_inventory': (102,)}
2024-06-07 03:50:13.509 | INFO     | networks:__init__:391 - Decoder CNN shapes: {'state_map': (16, 16, 166)}
2024-06-07 03:50:13.509 | INFO     | networks:__init__:392 - Decoder MLP shapes: {'state_inventory': (102,)}
2024-06-07 03:50:15.165 | INFO     | models:__init__:102 - Optimizer model_opt has 706924 variables.
2024-06-07 03:50:15.185 | INFO     | models:__init__:281 - Optimizer actor_opt has 71595 variables.
2024-06-07 03:50:15.186 | INFO     | models:__init__:292 - Optimizer value_opt has 98943 variables.


/home/wassname/miniforge3/lib/python3.9/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Dreamer(
  (_wm): OptimizedModule(
    (_orig_mod): WorldModel(
      (encoder): MultiEncoder(
        (_cnn): ConvEncoder(
          (layers): Sequential(
            (0): Conv2dSamePad(166, 16, kernel_size=(4, 4), stride=(2, 2), bias=False)
            (1): ImgChLayerNorm(
              (norm): LayerNorm((16,), eps=0.001, elementwise_affine=True)
            )
            (2): SiLU()
            (3): Conv2dSamePad(16, 32, kernel_size=(4, 4), stride=(2, 2), bias=False)
            (4): ImgChLayerNorm(
              (norm): LayerNorm((32,), eps=0.001, elementwise_affine=True)
            )
            (5): SiLU()
            (6): Conv2dSamePad(32, 64, kernel_size=(4, 4), stride=(2, 2), bias=False)
            (7): ImgChLayerNorm(
              (norm): LayerNorm((64,), eps=0.001, elementwise_affine=True)
            )
            (8): SiLU()
          )
        )
        (_mlp): MLP(
          (layers): Sequential(
            (Encoder_linear0): Linear(in_features=102, out_features=8, b

In [21]:
train_dataset??

Type:        generator
String form: <generator object from_generator at 0x7ea9049cef20>
Docstring:   <no docstring>

: 

- note model_opt includes actor.wm
  - encoder
  - rssm
  - heads
- actor

## Now lets play

In [7]:
assert state is not None
import numpy as np

# state

In [8]:
from tools import convert, add_to_cache
envs = train_envs
cache = train_eps

step, episode = 0, 0
done = np.ones(len(envs), bool)
length = np.zeros(len(envs), np.int32)
obs = [None] * len(envs)
agent_state = None
reward = [0] * len(envs)

indices = [index for index, d in enumerate(done) if d]
results = [envs[i].reset() for i in indices]
results = [r() for r in results]
for index, result in zip(indices, results):
    t = result.copy()
    t = {k: convert(v) for k, v in t.items()}
    # action will be added to transition in add_to_cache
    t["reward"] = 0.0
    t["discount"] = 1.0
    # initial state should be added to cache
    add_to_cache(cache, envs[index].id, t)
    # replace obs with done by initial state
    obs[index] = result
# step agents

In [9]:
# from tools.simulate

# step
# step, episode, done, length, obs, agent_state, reward = state
obs2 = {k: np.stack([o[k] for o in obs]) for k in obs[0] if "log_" not in k}
action, agent_state = agent(obs2, done, agent_state)

/media/wassname/SGIronWolf/projects5/worldmodels/dreamerv3-torch/nbs/../networks.py:792: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  ret = F.conv2d(
/media/wassname/SGIronWolf/projects5/worldmodels/dreamerv3-torch/.venv/lib/python3.9/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


2024-06-07 03:53:49.056 | INFO     | tools:write:85 - [2714] model_loss 7464.5 / model_grad_norm 4841.3 / state_map_loss 7449.7 / state_inventory_loss 7.5 / reward_loss 5.0 / cont_loss 0.1 / kl_free 1.0 / dyn_scale 0.5 / rep_scale 0.1 / dyn_loss 3.7 / rep_loss 3.7 / kl 3.7 / prior_ent 42.9 / post_ent 39.3 / normed_target_mean -0.0 / normed_target_std 0.0 / normed_target_min -0.0 / normed_target_max 0.0 / EMA_005 -0.0 / EMA_095 -0.0 / value_mean -0.0 / value_std 0.0 / value_min -0.0 / value_max -0.0 / target_mean -0.0 / target_std 0.0 / target_min -0.0 / target_max -0.0 / imag_reward_mean -0.0 / imag_reward_std 0.0 / imag_reward_min -0.0 / imag_reward_max -0.0 / imag_action_mean 21.2 / imag_action_std 12.6 / imag_action_min 0.0 / imag_action_max 42.0 / actor_entropy 3.6 / actor_loss -0.0 / actor_grad_norm 0.0 / value_loss 7.6 / value_grad_norm 8.4 / update_count 100.0 / fps 0.0


In [10]:
from torchinfo import summary

summary(agent, input=(obs, done, agent_state), depth=3)

Layer (type:depth-idx)                                            Param #
Dreamer                                                           --
├─OptimizedModule: 1-1                                            --
│    └─WorldModel: 2-1                                            --
│    │    └─MultiEncoder: 3-1                                     (84,592)
│    │    └─RSSM: 3-2                                             (270,848)
│    │    └─ModuleDict: 3-3                                       (351,484)
├─OptimizedModule: 1-2                                            --
│    └─ImagBehavior: 2-2                                          706,924
│    │    └─WorldModel: 3-4                                       (recursive)
│    │    └─MLP: 3-5                                              (71,595)
│    │    └─MLP: 3-6                                              (98,943)
│    │    └─MLP: 3-7                                              (98,943)
├─OptimizedModule: 1-3                        

In [11]:
summary(agent, input=(obs, done, agent_state), depth=4)

Layer (type:depth-idx)                                            Param #
Dreamer                                                           --
├─OptimizedModule: 1-1                                            --
│    └─WorldModel: 2-1                                            --
│    │    └─MultiEncoder: 3-1                                     --
│    │    │    └─ConvEncoder: 4-1                                 (83,680)
│    │    │    └─MLP: 4-2                                         (912)
│    │    └─RSSM: 3-2                                             128
│    │    │    └─Sequential: 4-3                                  (38,528)
│    │    │    └─GRUCell: 4-4                                     (99,072)
│    │    │    └─Sequential: 4-5                                  (16,640)
│    │    │    └─Sequential: 4-6                                  (50,432)
│    │    │    └─Linear: 4-7                                      (33,024)
│    │    │    └─Linear: 4-8                              

In [12]:
# agent._wm.heads

## Fine grained torchinfo

In [13]:
wm = agent._wm
data = next(agent._dataset) 
# self._train()
# post, context, mets = wm._train(data)
data = wm.preprocess(data)
embed = wm.encoder(data)
post, prior = wm.dynamics.observe(
    embed, data["action"], data["is_first"]
)

In [14]:
summary(wm.encoder, input_data=(data,), depth=4, col_names=["input_size", "output_size", "num_params", ])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
MultiEncoder                             [256, 32, 130, 110, 3]    [256, 32, 264]            --
├─ConvEncoder: 1-1                       [256, 32, 16, 16, 166]    [256, 32, 256]            --
│    └─Sequential: 2-1                   [8192, 166, 16, 16]       [8192, 64, 2, 2]          --
│    │    └─Conv2dSamePad: 3-1           [8192, 166, 16, 16]       [8192, 16, 8, 8]          (42,496)
│    │    └─ImgChLayerNorm: 3-2          [8192, 16, 8, 8]          [8192, 16, 8, 8]          --
│    │    │    └─LayerNorm: 4-1          [8192, 8, 8, 16]          [8192, 8, 8, 16]          (32)
│    │    └─SiLU: 3-3                    [8192, 16, 8, 8]          [8192, 16, 8, 8]          --
│    │    └─Conv2dSamePad: 3-4           [8192, 16, 8, 8]          [8192, 32, 4, 4]          (8,192)
│    │    └─ImgChLayerNorm: 3-5          [8192, 32, 4, 4]          [8192, 32, 4, 4]          --
│    │    │    └─Layer

In [15]:
# heads
feat = wm.dynamics.get_feat(post)
for name, head in wm.heads.items():
    try:
        o = summary(head, input_data=(feat,), depth=3, col_names=["input_size", "output_size", "num_params", ])
        print(name)
        print(o)
    except Exception as e:
        print(f"Summary Failed for {name} {e}")
        continue

decoder
Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
MultiDecoder                             [256, 32, 384]            --                        --
├─ConvDecoder: 1-1                       [256, 32, 384]            [256, 32, 16, 16, 166]    --
│    └─Linear: 2-1                       [256, 32, 384]            [256, 32, 256]            (98,560)
│    └─Sequential: 2-2                   [8192, 64, 2, 2]          [8192, 166, 16, 16]       --
│    │    └─ConvTranspose2d: 3-1         [8192, 64, 2, 2]          [8192, 32, 4, 4]          (32,768)
│    │    └─ImgChLayerNorm: 3-2          [8192, 32, 4, 4]          [8192, 32, 4, 4]          (64)
│    │    └─SiLU: 3-3                    [8192, 32, 4, 4]          [8192, 32, 4, 4]          --
│    │    └─ConvTranspose2d: 3-4         [8192, 32, 4, 4]          [8192, 16, 8, 8]          (8,192)
│    │    └─ImgChLayerNorm: 3-5          [8192, 16, 8, 8]          [8192, 16, 8, 8]          (32)
│    │

In [16]:
# fail as no call method
# summary(wm.dynamics, input_data=(embed, data["action"], data["is_first"]), depth=3, col_names=["output_size", "num_params", ])

In [17]:
actor = agent._task_behavior.actor

summary(actor.layers, input_data=(feat,), depth=3, col_names=["output_size", "num_params", "output_size" ])



Layer (type:depth-idx)                   Output Shape              Param #                   Output Shape
Sequential                               [256, 32, 128]            --                        [256, 32, 128]
├─Linear: 1-1                            [256, 32, 128]            (49,152)                  [256, 32, 128]
├─LayerNorm: 1-2                         [256, 32, 128]            (256)                     [256, 32, 128]
├─SiLU: 1-3                              [256, 32, 128]            --                        [256, 32, 128]
├─Linear: 1-4                            [256, 32, 128]            (16,384)                  [256, 32, 128]
├─LayerNorm: 1-5                         [256, 32, 128]            (256)                     [256, 32, 128]
├─SiLU: 1-6                              [256, 32, 128]            --                        [256, 32, 128]
Total params: 66,048
Trainable params: 0
Non-trainable params: 66,048
Total mult-adds (M): 16.91
Input size (MB): 12.58
Forward/backward p

In [18]:
value = agent._task_behavior.actor
summary(value.layers, input_data=(feat,), depth=3, col_names=["output_size", "num_params", "output_size" ])

Layer (type:depth-idx)                   Output Shape              Param #                   Output Shape
Sequential                               [256, 32, 128]            --                        [256, 32, 128]
├─Linear: 1-1                            [256, 32, 128]            (49,152)                  [256, 32, 128]
├─LayerNorm: 1-2                         [256, 32, 128]            (256)                     [256, 32, 128]
├─SiLU: 1-3                              [256, 32, 128]            --                        [256, 32, 128]
├─Linear: 1-4                            [256, 32, 128]            (16,384)                  [256, 32, 128]
├─LayerNorm: 1-5                         [256, 32, 128]            (256)                     [256, 32, 128]
├─SiLU: 1-6                              [256, 32, 128]            --                        [256, 32, 128]
Total params: 66,048
Trainable params: 0
Non-trainable params: 66,048
Total mult-adds (M): 16.91
Input size (MB): 12.58
Forward/backward p

In [19]:
8268

8268